# Convolutional Neural Networks

We'll explore convolutions and implement some convolutional neural networks. 

## Convolutions in 1D

We'll start by implementing a simple 1D convolution with a _rectangular kernel_ that works as a running average.

In [ ]:
import numpy as np

signal = np.array([1, 2, 3, 4, 5, 4, 3, 2, 1])

kernel = np.ones(4) / 4

averaged_signal = np.convolve(signal, kernel, mode="valid")

print(averaged_signal)

Try to modify the code above by changing the `signal` and the `kernel`.
For example, use the following kernels:

- _Prewitt kernel_ `[1, 0, -1]` for edge detection or differentiation.

- _Gaussian kernel_ `[.25 .5 .75 .5 .25]` for smoothing.

## Convolutions in 2D

We'll now move on and implement a 2D convolution with a _rectangular kernel_ that works as a local averaging.

In [ ]:
import numpy as np
from scipy import signal

signal = np.array([
    [1, 2, 3], 
    [4, 5, 6], 
    [7, 8, 9]
])

kernel = np.ones((3, 3)) / 9

output = signal.convolve2d(signal, kernel, mode="full")

print(output)

Tray modifying the code above changing the `signal` and the `kernel`.
For example, use the following kernels:

- _Prewitt kernel_ to detect edges:
    ```python
    kernel = np.array([
        [-1, 0, 1],
        [-1, 0, 1],
        [-1, 0, 1]
    ])
    ```

- _Sobel kernel_ to detect edges:
    ```python
    kernel = np.array([
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ])
    ```

- _Gaussian kernel_ to smooth the image:
    ```python
    kernel = np.array([
        [.04, .08, .12, .08, .04], 
        [.08, .16, .24, .16, .08], 
        [.12, .24, .36, .24, .12], 
        [.08, .16, .24, .16, .08], 
        [.04, .08, .12, .08, .04]
    ])
    ```

## Convolutional Layers

We'll now implement a convolutional layer in PyTorch with one input channel (`in_channels=1`), three output channels (`out_channels=3`), and a square kernel with size $3 \times 3$ (`kernel_size=3`, which is equivalent to `kernel_size=(3, 3)`).

We then initialize its weights to perform a local averaging, an horizonthal edge detection, and a vertical edge detection.

In [ ]:
import torch
import torch.nn as nn

model = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=3)

filters = torch.zeros(model.out_channels, model.in_channels, *model.kernel_size)
filters[0, 0, :, :] = torch.Tensor([[1, 1, 1], [1, 1, 1], [1, 1, 1]]) / 9
filters[1, 0, :, :] = torch.Tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
filters[2, 0, :, :] = torch.Tensor([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])
model.weight = nn.Parameter(filters)

L = 16
S = 4
image = torch.zeros(1, 1, L, L)
for idx in range(0, L, S):
    for idy in range(0, L, S):
        image[0, 0, idx:idx + S, idy:idy + S] = torch.randn(1)

features = model(image)

def plot_channel(channel):
    import matplotlib.pyplot as plt
    
    plt.figure(figsize=(4, 4))
    plt.imshow(channel, cmap="gray", aspect="equal", 
               extent=[0, channel.shape[0], 0, channel.shape[1]])
    plt.grid(color="red", linewidth=.5)
    plt.xticks(range(1, channel.shape[0] + 1))
    plt.yticks(range(1, channel.shape[1] + 1))
    plt.show() 

plot_channel(image.squeeze())
plot_channel(features[0, 0, :, :].detach().squeeze())
plot_channel(features[0, 1, :, :].detach().squeeze())
plot_channel(features[0, 2, :, :].detach().squeeze())

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 4, figsize=(16, 4))

def plot_channel(ax, title, channel, offset=0):
    ax.imshow(channel, cmap="gray", aspect="equal", 
               extent=[0, channel.shape[0], 0, channel.shape[1]])
    ax.grid(color="red", linewidth=.5)
    ax.set_title(title)
    ax.set_xticks(range(0, channel.shape[0] + 1))
    ax.set_xlim(- offset, channel.shape[0] + offset)
    ax.set_yticks(range(0, channel.shape[1] + 1))
    ax.set_ylim(- offset, channel.shape[1] + offset)

plot_channel(axs[0], "image", image.squeeze())

plot_channel(axs[1], "feature 0", features[0, 0, :, :].detach().squeeze(), 1)
plot_channel(axs[2], "feature 1", features[0, 1, :, :].detach().squeeze(), 1)
plot_channel(axs[3], "feature 2", features[0, 2, :, :].detach().squeeze(), 1)

plt.tight_layout()
plt.show()